## SQL Alchemy run through

In the previous notebook, we made an RDS Instance that contains all of our tables. In this subsection, we will go over retrieving this information using SQL Alchemy

## Making the engine

In [ ]:
from sqlalchemy import create_engine, select
from sqlalchemy.dialects.postgresql import psycopg2
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
import psycopg2
import sqlalchemy

In [ ]:
import boto3
import pandas as pd

In [ ]:
int(sqlalchemy.__version__[0]) >= 1

In [ ]:
from sqlalchemy.sql import and_, or_

In [ ]:
from sqlalchemy import func, text, case

In [ ]:
from pprint import pprint

### first get the rds instance we deployed earlier

In [ ]:
rds = boto3.client('rds')

In [ ]:
def getAddressAndPort(instance_identifier):
    import boto3
    rds_client = boto3.client('rds')

    instance =filter(
        lambda instance: instance['DBInstanceIdentifier'] == instance_identifier,
                           rds.describe_db_instances()['DBInstances'])
    endpoint= list(instance)[0]['Endpoint']
    address, port = endpoint['Address'], endpoint['Port']
    return address, port



In [ ]:
getAddressAndPort('cfsdb')

In [ ]:
instances = list(filter(lambda instance: instance['DBInstanceStatus']=='available', rds.describe_db_instances()['DBInstances']))

In [ ]:
instance = instances[0]

In [ ]:
instances

In [ ]:
address, port = instance['Endpoint']['Address'], instance['Endpoint']['Port']
db_name = instance['DBName']
username = instance['MasterUsername']
password = 'TestPassword'

In [ ]:
address, db_name

In [ ]:
engine = create_engine('postgresql+psycopg2://{username}:{password}@{add}:{port}/{db_name}'.format(
    username=username,
    password=password,
    add=address,
    port=port,
    db_name=db_name), echo=True)

In [ ]:
conn = engine.connect()

In [ ]:
# at this point we can remove entry 0 that we used in the earlier notebook
conn.execute("""
             DELETE FROM ONLY transactions
             WHERE "SHIPMT_ID" = 0 """)

In [ ]:
assert conn.execute("""
    SELECT  "SHIPMT_ID" 
    FROM transactions
    LIMIT 1
    """).fetchall()[0][0] == 1, "Failed to delete row"

In [ ]:
# first try some basic sql queries
# get hazardous or not hazardous
q = conn.execute("""
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             """
             )

In [ ]:
q.fetchall()

### Now that you have it working, use SqlAlchemy to show all the tables you currently have

MetaData allows you to store all the information about all the tables in your database in a convenient location you have to use the reflect method of MetaData to accomplish this.

In [ ]:
meta = MetaData()

In [ ]:
meta.reflect(engine)

In [ ]:
for table in meta.tables:
    print(table)

In [ ]:
transactions = meta.tables['transactions']

state_latlon = meta.tables['state_latlon']

cfs_areas    = meta.tables['cfs_areas']
FIPS_States  = meta.tables['FIPS_States']
transport_mode = meta.tables['transport_mode']
sctg         = meta.tables['sctg']
transactions_joined = meta.tables['transactions_joined']

In [ ]:
s = select([sctg])
r = conn.execute(s)

In [ ]:
for row in r:
    print(row[sctg.c.Description], row[sctg.c.SCTG])
r.close()

### Joins

In [ ]:
# join fips states with avg lat lon
for row in conn.execute(select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))):
    print(row)

In [ ]:
FIPS_States.columns.get('Alpha code')

### operators
SQL Alchemy converts python predicates to sql predicates. So you can describe everything in a pythonic sense.

In [ ]:
print((FIPS_States.c.get('Alpha code') == 'KK').compile())
print(((FIPS_States.c.get('Alpha code') < 'CA') & ('NY' < FIPS_States.c.get('Alpha code'))).compile())

In [ ]:
transactions.columns.keys()

In [ ]:
for row in conn.execute(select([transactions]).limit(10)).fetchall():
    print(row.items())

In [ ]:
# to get a list of dicts per row
get_row = lambda row: (dict(row.items()))
get_rows = lambda row_list: map(get_row, row_list)

list_of_dicts = [dict(row.items()) for row in conn.execute(select([transactions]).limit(10)).fetchall()]

### Conjunctions
Conjunctions can be used to perform and, or, not operations in sql.

In [ ]:
# get all states that lie above the 35N, and less than -104 longitude

# first join the fips and state_latlons
j = FIPS_States.join(
    state_latlon, 
    onclause= FIPS_States.c.get("Alpha code") == state_latlon.c.state)

# now select from this join
s = (select([FIPS_States, state_latlon])
     .where(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        )
    ).select_from(j)
)

In [ ]:
conn.execute(s).fetchall()

In [ ]:
# now get those along with all the states in the bottom right corner of the US
# delinieated by below 35N and right of 88W
s = (select([FIPS_States, state_latlon])
     .where(or_(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        ),
        and_(
            state_latlon.c.latitude < 35,
            state_latlon.c.longitude > -88)
        )
    ).select_from(j)
)

In [ ]:
pprint(str(s))

In [ ]:
pprint(str(select([text('sub."Alpha code"')]).select_from(s.alias('sub'))))

In [ ]:
from sqlalchemy import desc
from sqlalchemy.sql import literal_column

In [ ]:
# using text to select subqueries
# instead of text, we use literal_column so that we can use the label method
conn.execute(select([literal_column('sub."Alpha code"').label("alpha code")]).select_from(s.alias('sub'))
            .order_by(desc('alpha code'))).fetchall()

#### Binding parameters

In [ ]:
from sqlalchemy.sql import bindparam, func

In [ ]:
get_results = lambda s, **kwargs: conn.execute(s, kwargs).fetchall()

In [ ]:
get_results(
    select([FIPS_States])
        .where(or_(
              FIPS_States.c["Alpha code"] == bindparam('from_'),
              FIPS_States.c["Alpha code"] == bindparam('to_')
            )
        ),
    from_ = "CA",
    to_   = "TX"
)

#### Functions

Functions are created using the func keyword.  These are translated to sql function queries.

func.myFunc(transactions.c.SHIPMT_WEIGHT) translates to 

    myFunc(transactions."SHIPMT_WEIGHT")

In [ ]:
print(func.myFunc([transactions.c.SHIPMT_VALUE]).compile().params)

In [ ]:
conn.execute(select([func.max(transactions.c.SHIPMT_VALUE).label("Max Val")])).scalar()

#### Window functions
Chain over with functions

In [ ]:
list(get_rows(get_results(select([transactions,
                                 func.sum(transactions.c.SHIPMT_VALUE).over(
                                    order_by=transactions.c.SHIPMT_ID)
                                  .label("SUM")
                                 ]).limit(10))))

In [ ]:
from sqlalchemy.sql import union

In [ ]:
u = union(
    sctg.select().where(sctg.c.SCTG < 3),
    sctg.select().where(sctg.c.SCTG > 10)
    )

conn.execute(u).fetchall()

In [ ]:
query_s = (select([func.count(transactions.c.SHIPMT_ID)])
           .where(transactions.c.SCTG).as_scalar())

In [ ]:
conn.execute(select([query_s]))

In [ ]:
conn.execute(u).fetchone()

##### Correlated Subqueries

When an enclosed query depends on a table from the enclosing context, it does not need to explicitly provide a FROM clause

In [ ]:
stmt = (select([transactions, FIPS_States.c['Alpha code']])
        .select_from(
        transactions.join(FIPS_States, onclause=transactions.c.ORIG_STATE == FIPS_States.c["Numeric code"]))
        .where(FIPS_States.c['Alpha code'].like('C%'))
        )

In [ ]:
enclosing_stmt = select([transactions.c.SCTG]).select_from(stmt).limit(10)

In [ ]:
conn.execute(enclosing_stmt)

In [ ]:
transactionsBetweenStates = (select([transactions_joined])
                                 .where(
                transactions_joined.c.orig_alpha_code==bindparam("orig_code"))
                                  .where(
                transactions_joined.c.dest_alpha_code==bindparam("dest_code"))
)


In [ ]:
stmt = (select([transactionsBetweenStates.c['SHIPMT_VALUE']])
.select_from(transactionsBetweenStates.alias())
)



In [ ]:
get_rows(conn.execute(stmt, orig_code="TX", dest_code="WA").fetchall())

##### Controlling correlations

You can use the correlate method to control the behavior of correlations

In [ ]:
import json

Now you can export to a json file for a temporary data file

In [ ]:
#with open('app/testdata.json', 'w') as f:
#    f.write((json.dumps(list_of_dicts, indent=2, separators=(',', ':'))))

In [ ]:
c = case([(transactions.c['ORIG_STATE']==transactions.c['DEST_STATE'], True)], else_=False)

In [ ]:
s = select([transactions]).where(c).limit(10);

In [ ]:
for i in get_rows(conn.execute(s).fetchall()):
    print(i)

Data structure

{orig_state_code:
 dest_state_code:
 orig_coords:
 dest_coords:
}

In [ ]:
joined_states = text(
    """
    SELECT 
        fips."Name" as state_name,
        fips."Numeric code" as numeric_code,
        state_coords.latitude as lat,
        state_coords.longitude as long
    FROM "FIPS_States" fips
    JOIN "state_latlon" state_coords
    ON state_coords.state = fips."Alpha code"
    """)

list(get_rows(conn.execute(joined_states).fetchall()))

In [ ]:
states_and_coords = select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))

In [ ]:
conn.execute(states_and_coords)

In [ ]:
orig_state = 'CA'
dest_state = 'TX'

In [ ]:
states = select([FIPS_States]).where(or_(
    FIPS_States.c['Alpha code']==orig_state,
    FIPS_States.c['Alpha code']==dest_state)
)

conn.execute(states).fetchall()

### I need a way to query a database like so, 

    "SELECT * 
        FROM transactions
        WHERE orig_state = 'CA' AND dest_state= 'TX'
    "

In [ ]:
d = pd.read_sql_query(con = engine, sql = select([FIPS_States]))

In [ ]:
d2 = pd.read_sql_query(con = engine, sql = select([transactions]).limit(2000))

In [ ]:
d3 = pd.merge(pd.merge(d, d2, left_on=["Numeric code"], right_on=["ORIG_STATE"]), d, 
         left_on="DEST_STATE", right_on="Numeric code")

In [ ]:
d3 = d3.rename(columns={'Name_x': 'Source_Name', 
                   'Alpha code_x': 'Source_alpha_code',
                    'Name_y': 'Dest_Name',
                    'Alpha code_y': 'Dest_alpha_code'})

Use set operators from python to get the intersection of these predicates

In [ ]:
d3[(d3['Dest_alpha_code'] == 'CA') & (d3['Source_alpha_code'] == 'TX')]

In [ ]:
gb = d3.groupby(['Dest_alpha_code', 'Source_alpha_code'])

In [ ]:
gb.get_group(('CA', 'CA')).SHIPMT_VALUE.describe().reset_index()

In [ ]:
gb.groups.keys()

### This way works, but I'm worried that there will be a lot of overhead joining tables constantly, so I will create a new table that contains the source name, and alpha code along with the destination info into a new table called transactions_joined

### You can create a new table from a previous one using sql alchemy's Insert From Select

In [ ]:
#list(get_rows(conn.execute(select([FIPS_States.c.get("Numeric code")])).fetchall()))

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

In [ ]:
orig = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.ORIG_STATE == FIPS_States.c.get("Numeric code"))


s = select([transactions, 
            FIPS_States.c.get("Alpha code").label("orig_state_alpha_code"),
            FIPS_States.c.Name.label("orig_state_name"),
            state_latlon.c.latitude.label('orig_lat'),
            state_latlon.c.longitude.label('orig_lon')
            ]).select_from(orig).limit(10)

list(get_rows(conn.execute(s).fetchall()))

In [ ]:
dest = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.DEST_STATE == FIPS_States.c.get("Numeric code"))

new_table =  (select([s.alias(), 
            FIPS_States.c.get("Alpha code").label("dest_state_alpha_code"),
            FIPS_States.c.Name.label("dest_state_name"),
            state_latlon.c.latitude.label('dest_lat'),
            state_latlon.c.longitude.label('dest_lon')])
            .select_from(dest).limit(10))

In [ ]:
list(get_rows(conn.execute(select([new_table.alias()])).fetchall()))

In [ ]:
meta.tables['FIPS_States']

In [ ]:
t = text("""
    SELECT transactions.*,
        orig."Name" as orig_name,
        orig."Alpha code" as orig_alpha_code,
        orig."latitude" as orig_lat,
        orig."longitude" as orig_long,
        dest."Name" as dest_name,
        dest."Alpha code" as dest_alpha_code,
        dest."latitude" as dest_lat,
        dest."longitude" as dest_lon
    FROM transactions
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) orig
        ON orig."Numeric code" = transactions."ORIG_STATE"
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) dest
        ON dest."Numeric code" = transactions."DEST_STATE"
    WHERE
        dest."Alpha code"=:dest_code AND orig."Alpha code" =:orig_code
    LIMIT 100
        
    """)

In [ ]:
select([transactions,
        ])

In [ ]:
conn.execute(t.params(dest_code="CA", orig_code="TX")).fetchone()

In [ ]:
#with open('test_data.json', 'w') as f:
#    f.write(
#        json.dumps(
#            list(get_rows(conn.execute(t).fetchall())),
#            indent=2,
#            separators=(",", ":")
#             ))

In [ ]:
from sqlalchemy.sql import table

In [ ]:
#new_table = new_table.alias("namedStates")

In [ ]:
new_table_t = table(new_table)

In [ ]:
new_tableb

In [ ]:
orig_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.orig_state_alpha_code)

dest_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.dest_state_alpha_code)

In [ ]:
new_table2 = (select([new_table.alias(),
                    state_latlon.c.latitude.label('orig_lat'),
                    state_latlon.c.longitude.label('orig_lon')])
            .select_from(orig_.alias()))

In [ ]:
get_rows(conn.execute(select([new_table2.alias()])))

In [ ]:
list(get_rows(conn.execute(select([state_latlon])).fetchall()))

In [ ]:
list(get_rows(conn.execute(select([FIPS_States])).fetchall()))

In [ ]:
orig = state_latlon_joined.alias("orig")

dest = state_latlon_joined.alias("dest")
   

In [ ]:
orig.c.keys()

In [ ]:
from sqlalchemy.sql import bindparam

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

stmt = (select([transactions,
              orig.c.FIPS_States_Name.label("orig_name"),
              orig.c["FIPS_States_Alpha code"].label("orig_alpha_code"),
              dest.c["FIPS_States_Name"].label("dest_name"),
              dest.c["FIPS_States_Alpha code"].label("dest_alpha_code"),
              orig.c.state_latlon_latitude.label("orig_lat"),
              orig.c.state_latlon_longitude.label("orig_lon"),
              dest.c.state_latlon_latitude.label("dest_lat"),
              dest.c.state_latlon_longitude.label("dest_lon")])
        .select_from(transactions
                    .join(orig, transactions.c.ORIG_STATE == orig.c["FIPS_States_Numeric code"])
                    .join(dest, transactions.c.DEST_STATE == dest.c["FIPS_States_Numeric code"]))
        )
        #.where(orig.c["FIPS_States_Alpha code"] == bindparam("orig_code")))
        #.where(dest.c["FIPS_States_Alpha code"] == bindparam("dest_code")))

In [ ]:
list(get_rows(conn.execute(stmt.order_by(desc(transactions.c.SHIPMT_VALUE)).limit(10), orig_code='CA').fetchall()))

Create table from another table


In [ ]:
stmt.columns.items()

In [ ]:
# a way to copy a columns from one table to another
t = Table("transactions_joined", meta, *(map(lambda c: c.copy(), stmt.columns)))


In [ ]:
t.create(bind=engine)

In [ ]:
# pass in the column keys and the statement to select from
conn.execute(t.insert().from_select(stmt.columns.keys(), stmt))

In [ ]:
meta.reflect(engine)

In [ ]:
meta.tables.keys()

In [ ]:
tj = meta.tables['transactions_joined']

In [ ]:
q = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
     .where(tj.c.dest_alpha_code==bindparam("dest_code"))
     .order_by(tj.c.SHIPMT_VALUE)
     .limit(10))

q2 = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
      .order_by(desc(tj.c.SHIPMT_VALUE))
      .limit(10))
list(get_rows(conn.execute(q, orig_code="CA", dest_code="TX")))

In [ ]:
tj.columns.keys()

In [ ]:
from sqlalchemy import Index

Index('orig_fip', transactions.c.ORIG_STATE)

In [ ]:
import timeit

I want to experiment with adding an index and testing the response of the query

In [ ]:
import time

In [ ]:
print("We will be running: {}".format(str(q)))

We need to make an index on the orig_alpha_code and dest_alpha_code

In [ ]:
index = text("CREATE INDEX ALPHA_INDEX ON transactions_joined "\
             "(orig_alpha_code, dest_alpha_code)")

In [ ]:
conn.execute(index)

In [ ]:
meta.tables['transactions_joined']

In [ ]:
#http://stackoverflow.com/questions/5605019/listing-indices-using-sqlalchemy
from sqlalchemy.engine import reflection

In [ ]:
insp = reflection.Inspector.from_engine(engine)
indices = []
for name in insp.get_table_names():
    for index in insp.get_indexes(name):
        indices.append((name,index))

In [ ]:
indices

In [ ]:
import datetime

In [ ]:
now = datetime.datetime.now()
l = list(get_rows(conn.execute(q, orig_code="NY", dest_code="TX").fetchall()))
diff = datetime.datetime.now() - now



In [ ]:
print("It took {} seconds to execute the query".format(diff))
pprint(l)

In [ ]:
#http://docs.sqlalchemy.org/en/latest/faq/performance.html#query-profiling
from sqlalchemy import event
from sqlalchemy.engine import Engine

In [ ]:
@event.listens_for(Engine, "before_cursor_execute")
def before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    conn.info.setdefault('query_start_time', []).append(time.time())
    print("Start Query: {}".format(statement))

In [ ]:
@event.listens_for(Engine, "after_cursor_execute")
def after_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    total = time.time() - conn.info['query_start_time'].pop(-1)
    print("Query Done\nTotal-Time: {}".format(total))

In [ ]:
l = list(get_rows(conn.execute(q2, orig_code="CA").fetchall()))

In [ ]:
cmp = q.params(orig_code="CA", dest_code="TX").compile()

In [ ]:
print(cmp)

In [ ]:
# try creating a new index on shipmt_value
i = Index('shipmt_value', tj.c.SHIPMT_VALUE)
i.create(engine)

In [ ]:
l = list(get_rows(conn.execute(q, orig_code="WY", dest_code="WY").fetchall()))

In [ ]:
# that index made it much faster
# let's try adding a functional index as well, a desc version of shipmt_value
i = Index('shipmt_value_desc', tj.c.SHIPMT_VALUE.desc())
i.create(engine)

### Adding indexes greatly reduces time!

## Now I will be figuring out the queries needed for the other data views

In [ ]:
# we want to get the sctg names joined
q = (select([tj.c.SCTG, func.count(tj.c.SHIPMT_ID).label("counts")])
     .where(and_(
        tj.c.dest_alpha_code == 'TX',
        tj.c.orig_alpha_code == 'WY'))
     )
        

In [ ]:
g1 = q.group_by(tj.c.SCTG)

In [ ]:
for i in get_rows(conn.execute(g1.limit(10)).fetchall()):
    print(i)

In [ ]:
sctg.columns.keys()

In [ ]:
for i in get_rows(conn.execute(select([g1.alias(), sctg]).where(tj.c.SCTG == cast(sctg.c.SCTG, TEXT()))).fetchall()):
    print(i)

In [ ]:
from sqlalchemy import cast, Numeric, String, TEXT

In [ ]:
tj.c.SCTG, print(cast(sctg.c.SCTG, TEXT()))